# Simple inheritance

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

X = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)
y = np.array([2, 3, 4, 5, 6])

class Child(LinearRegression):
    def __init__(self):
        super().__init__()
    def fit(self, X, y, *args, **kwargs):
        print("I am a child class")
        return super().fit(X, y, *args, **kwargs)
    
reg = Child()
reg.fit(X, y)

I am a child class


Child()

# Swapping the class

In [2]:
reg2 = LinearRegression()
reg2.__class__ = Child
reg2.fit(X,y)

I am a child class


Child()

# Basic monkey patching 

In [3]:
def my_fit(self, *args, **kwargs):
    print("I'm just a monkey patch!")

LinearRegression.fit = my_fit
reg3 = LinearRegression()
reg3.fit(X, y)

I'm just a monkey patch!


# Dynamic monkey patching

In [4]:
# A top-level class from the library from whose internal workings you want to extract something
class SomeLibraryClass:
    def __call__(self, X, y):
        # The instantiation of LinearRegression is happening deep in code of your dependency
        # Here it's just one call deep, but could be many levels
        reg = LinearRegression().fit(X, y)
        return reg


def my_function_that_extracts_internal_value(X, y):
    
    # Dynamically create a function that stores an internal value in our local variable
    container = {}
    def function_that_creates_a_value_and_throws_it_away(self, X, y):
        # Imagine some complicated calculations leading up to that
        intermediate_value_to_throw_away = np.median(y)
        print("The value I would throw away:", intermediate_value_to_throw_away)
        container["value"] = intermediate_value_to_throw_away
        # imagine some more complicated calculations here that the original function did
        return np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))
    
    # And monkey patch a method of the library class with it
    LinearRegression.fit = function_that_creates_a_value_and_throws_it_away
    
    # Call the top-level thing that somewhere in its guts calls the class we monkeypatched
    tmp = SomeLibraryClass()
    tmp(X, y)
    
    # The "container" variable now contains the internal value we wanted to extract
    return container["value"]

# Now call my 
my_function_that_extracts_internal_value(X,y)
        

The value I would throw away: 4.0


4.0